In [ ]:
cd ../..

In [ ]:
import asyncio
import glob
import pathlib
import threading
from typing import Callable, Mapping

import numpy as np
from chess import engine, pgn

from scripts import bitboard, tanh

In [ ]:
async def worker(path: pathlib.PurePath, semaphore: asyncio.Semaphore, *, checkpoint: int = 10000) -> Mapping[str, list]:
    async with semaphore:
        kwds = {"X": [], "y": []}
        _, simple_engine = await engine.popen_uci(
            "lib/stockfish/stockfish"
        )
        with open(path) as f:
            i = 0
            save_checkpoint = lambda: np.savez(
                f"data/npz/{path.stem}", **kwds
            )
            while True:
                if not (game := pgn.read_game(f)):
                    break
                if mainline := game.mainline():
                    board = np.random.choice(list(mainline)).board()
                    kwds["X"].append(
                        bitboard(board)
                    )
                    kwds["y"].append(
                        tanh(
                            (await simple_engine.analyse(
                                board,
                                engine.Limit(time=0.1)
                            ))["score"].relative.score(mate_score=7625), k=0.0025
                        )
                    )
                    i += 1
                    if i % checkpoint == 0:
                        save_checkpoint()
        await simple_engine.quit()
        save_checkpoint()
        return kwds

In [ ]:
async def main() -> None:
    semaphore = asyncio.Semaphore(3)
    await asyncio.gather(*(
        asyncio.ensure_future(
            worker(pathlib.PurePath(file), semaphore)
        ) for file in glob.glob("data/*.pgn")
    ))

In [ ]:
def run(main: Awaitable[Any], *, debug: bool = False) -> None:
    asyncio.set_event_loop_policy(
        engine.EventLoopPolicy()
    )
    asyncio.run(
        main,
        debug=debug
    )

In [ ]:
threading.Thread(
    target=run,
    args=(main(),)
).start()